## Compare BioTac data of everyday touch interactions against actuator feedback

<b>Setup</b>: 
1. BioTac bonded to finger via tape and act like a dummy finger (Is two BioTac possible? Is there a way to bond BioTac with pretty visual appearance)
2. Use a camera on the side to record the touch interaction with the BioTac finger
3. Force: light->BioTac DC pressure < 2kPa, medium-to-high: BioTac DC pressure ~= 4kPa

<b>[Totle time] Touch Interactions to be Measured (with each repeated between 6 to 12 times over 1 minute, with ~2 seconds interval)</b>: 
1. [1 min] Wear the BioTac to lightly tap on a keyboard, with very light force just enough to press down a key, and with movement as natural as possible (Maybe type "max planck"?)
2. [1 min] With one BioTac bonded to the index finger (if possible, bond another one to the thumb), then pinch grasp and lift a pen from a table, with force just enought to lift the pen, and as natural as possible, and then drop it back. 
3. [1 min] With the same BioTac setup and gesture described above, lift a small fruit (diameter < 5 cm, grape/berry) from the table.
4. [1 min] Press one BioTac on the forearm of the resting arm
5. [1 min] Pinch Grasp one finger of the resting hand using two BioTac
6. [1 min] Press on a hinging coat (or a pile of fabric)
7. [1 min] Pinch Grasp a hinging coat (or a pile of fabric)
8. [1 min] Lifting a water bottle (or any graspable object around 0.5 kg), with a medium-to-high force level. 

<b>Touch Interactions used in VR demos</b>: Touch animal, body tissue, clothes, keypad, grape/bubble
<img src="VRTouchInteractionReference.png">

In [ ]:
'''
Visualize all results of pressure and impedance measurement of BioTac for producing Figure 3
Author: Yitian Shao
Created on 2023.03.08
'''

from os import walk
import os.path as ospa
import numpy as np
import re
import matplotlib.pyplot as plt
from matplotlib.pyplot import cm
from matplotlib.patches import Arc
import pandas as pd
from scipy import signal
from scipy.interpolate import griddata
import seaborn as sns

# from IPython.display import set_matplotlib_formats
# set_matplotlib_formats('svg', 'pdf')

plt.rc('font', size=10, family='Verdana') # 'Tahoma', 'DejaVu Sans', 'Verdana'"
plt.rc('axes', edgecolor='0.5', linewidth=0.75)
plt.rc('axes.spines', **{'bottom':True, 'left':True, 'right':False, 'top':False})
plt.rcParams.update({'errorbar.capsize': 4})
# plt.rcParams['figure.dpi'] = 300
# plt.rcParams['savefig.dpi'] = 300

figSize_inch = (3.2, 2.4)

Fs = 100

In [ ]:
'''
General Functions
'''
def unifyAxesColor(ax, color='k'):
    ax.spines['top'].set_color(color)
    ax.spines['left'].set_color(color)
    ax.spines['right'].set_color(color)
    ax.spines['bottom'].set_color(color)
    ax.tick_params('both', colors='k')

def decodeActuatorInfo(rootName):
    actLabel = re.findall('Act\d+mm\d+\.\d+mL', rootName)
    if actLabel:
        actLabel = actLabel[0]
    return actLabel

def decodeData(fileName, numFormat, frontCode='', rearCode=''):
    segStr = re.findall(frontCode+numFormat+rearCode, fileName)
    if segStr:
        numData = float(re.findall(numFormat, segStr[0])[0])
    else:
        numData = None
    return numData

def lowpassSmooth(datain, cutFreqRatio = 0.05, order = 8):
    b, a = signal.butter(order, 2 * cutFreqRatio, btype='low')
    dataout = signal.filtfilt(b, a, datain)
    return dataout

def loadDataSegment(measureDataPath, root, fileName, lpFreq=0.5):
    actLabel = decodeActuatorInfo(root)
    if actLabel:
        tubeLen = decodeData(actLabel, '\d+', rearCode='mm')
        infillVol = decodeData(actLabel, '[\d+\.]*\d+', rearCode='mL')   

        vLevel = decodeData(fileName, '\d+', frontCode='v')
        cTime = decodeData(fileName, '\d+', frontCode='c')
        dTime = decodeData(fileName, '\d+', frontCode='d')
        trialNum = decodeData(fileName, '\d+', frontCode='t')
        dLabel = "L%03dF%.1fV%03dC%04dD%04d" % (tubeLen, infillVol, vLevel, cTime, dTime)  

        print("%s --- Len=%dmm, Infill=%.1fmL, Condi: v=%d%% c=%dms d=%dms t=%d" % 
              (dLabel, tubeLen, infillVol, vLevel, cTime, dTime, trialNum))         

    ''' Read in data '''
    data = np.genfromtxt(ospa.join(measureDataPath, root, fileName), delimiter=',')

    t = data[:,0]
    pDC = (data[:,1] - data[0,1]) * 0.0365 # DC Pressure = (Pdc - Offset) 0.0365 kPa/bit
    tAC = data[:,2]
    tDC = data[:,3]
    eData = (4095/data[:,4:] - 1) * 10000 # (Unit: Ohm) Impedance = (4095/En - 1) 10 kOhm
    
    ''' Smooth impedance signal when 0 < lpFreq < 0.5, otherwise disabled'''
    if lpFreq < 0.5 and lpFreq > 0:
        for i in range(19):
            eData[:,i] = lowpassSmooth(eData[:,i], cutFreqRatio=lpFreq, order = 8)
            
    eData = eData - np.mean(eData[0:20,:], axis=0) # Subtract DC (the first 0.2 sec signal)

    outData = {}
    outData['t'] = t
    outData['pDC'] = pDC
    outData['tAC'] = tAC
    outData['tDC'] = tDC
    outData['eData'] = eData
    
    return outData

def examData(data, tInstance=1.5, tRange=[0, 30], dispTem=False):
    ''' Exam data '''
    ti = np.argmax(data['t']>=tInstance)
    print("Selected time instance at %.2f sec, index = %d" % (tInstance, ti))
    
    if dispTem: # Display temperature readings in addition to the dc pressure reading
        _,(ax1, ax2, ax3)  = plt.subplots(3, 1, dpi=300, figsize=(3,2))
        ax2.plot(data['t'], data['tAC'])
        ax2.set_xlim(tRange)
        ax3.plot(data['t'], data['tDC'])
        ax3.set_xlim(tRange)
    else:
        _,ax1 = plt.subplots(dpi=300, figsize=(3,1))
        
    ax1.plot(data['t'], data['pDC'])
    ax1.plot(data['t'][[ti, ti]], [0, 3], 'k')
    ax1.set_xlim(tRange)
    
    _, ax = plt.subplots(dpi=300, figsize=(3,2))
    for i in range(19):
        ax.plot(data['t'], data['eData'][:,i])
    ax.set_xlim(tRange)
    ax.plot(data['t'][[ti, ti]], [-0.1, 0.1], 'k')
    
def generateMapResult(tind, btData, yMax=5, unifyRange=None, alpha=4000, reverseColorbar=True):    
# cmap=cm.get_cmap('inferno', 100) # cmap=cm.get_cmap('binary', 100) # cmap=cm.get_cmap('RdYlBu', 100)
    colorr = ""
    if reverseColorbar:
        colorr = "_r"
    cmap=cm.get_cmap("viridis"+colorr, 100)

    btMap = BiotacMap()
    btMap.initializeDistanceMap(alpha=alpha)

    frameNum = len(tind)
    
    ''' Remove DC of electrode measurement '''
    eData = btData['eData']
    eData = eData - np.mean(eData[tind[0]:tind[1],:], axis=0) 

    fig0, axes = plt.subplots(1, frameNum, dpi=300, figsize=figSize_inch)
    for i in range(frameNum):
        btMap.mapFromCubicInterp(axes[i], eData[tind[i],:])
        axes[i].set_title("%.2fs" % btData['t'][tind[i]], size=3)

    ''' Smooth interpolation of impedance map '''
    mapValues = []
    mapMins = []
    mapMaxs = []
    for i in range(frameNum):
        mapValue = btMap.constructMap(eData[tind[i],:])
        mapValues.append(mapValue)
        mapMins.append(np.amin(mapValue))
        mapMaxs.append(np.amax(mapValue))
    
    if unifyRange is None:
        unifyRange = [min(mapMins), max(mapMaxs)]

    fig1, ax0 = plt.subplots(dpi=300, figsize=(3,1))
    fig1cbar, cbarax = plt.subplots(dpi=300, figsize=(1,1))
    btMap.dispMaps(ax0, mapValues, xShift=300, cbarax=cbarax, cmap=cmap, unifyRange=unifyRange, s=0.001, dispOutline=False)

    ''' Plot Signal waveform '''  
    fig2, ax1 = plt.subplots(dpi=300, figsize=(3,1))
    ax1.plot(btData['t'], btData['pDC'])

    for i in range(frameNum):
        ax1.plot(btData['t'][[tind[i], tind[i]]], [0, yMax], 'tab:grey', lw=0.5)

    ax1.set_xlim([btData['t'][tind[0]]-0.1, btData['t'][tind[-1]]+0.1]);
    ax1.set_ylim([-0.1, yMax])
    unifyAxesColor(ax1, color='k')

    ax1.set_xlabel("Time (secs)")
    ax1.set_ylabel("Pressure (kPa)");
    
    return fig1, fig1cbar, fig2, ax1

In [ ]:
'''
Class
'''
class BiotacMap:
    ''' Class of biotac mapping for impedance data visualization '''
    def __init__(self):
        ''' Biotac Electrode 1-19 layout '''
        eXY = [[278, 523], [226, 588], [226, 639], [278, 666], [226, 704], [278, 757],
               [174, 458], [213, 497], [135, 497], [174, 536], [69, 523], [122, 588],
               [122, 639], [69, 665], [122, 704], [69, 756], [174, 587], [174, 691], [174, 756]]
        eXY = np.array(eXY)
        eXY[:,1] = -eXY[:,1]+800

        self.eXY = eXY
        self.eNum = eXY.shape[0]
        
        extent = (50,300,0,400)
        self.extent = extent
        
        self.grid_x, self.grid_y = np.mgrid[extent[0]:extent[1], extent[2]:extent[3]]
        
        ''' Finger-shape mapping: Set electrode 10 eXY[9,:] as the arc center of the shape '''
        self.centerX = self.eXY[9,0]
        self.centerY = self.eXY[9,1]
        self.bottomY = self.eXY[18,1] - 20
        self.cRadius = 125

        ''' Biotac finger shape '''
        grid_x, grid_y = np.mgrid[(self.centerX-self.cRadius):(self.centerX+self.cRadius), 
                                  self.bottomY:(self.centerY+self.cRadius)]
        gridXY = np.vstack((grid_x.flatten(),grid_y.flatten())).T

        ind = (gridXY[:,1] <= self.centerY) | ((gridXY[:,1] > self.centerY) & 
              ((np.square(gridXY[:,0]-self.centerX) + np.square(gridXY[:,1]-self.centerY)) < (self.cRadius*self.cRadius)))
        self.gridXY = gridXY[ind,:]
        
    ''' Compute distance map '''    
    def initializeDistanceMap(self, alpha=100): # Alpha determine the smoothness of the interpolation: Phi = 1/(dist + Alpha)
        distMap = []
        for i in range(self.eNum):
            aDistance = np.sqrt(np.square(self.gridXY[:,0]-self.eXY[i,0]) + np.square(self.gridXY[:,1]-self.eXY[i,1]))
            distMap.append(aDistance)
            
        self.distMap = np.array(distMap)

        self.Phi = 1 / (np.square(self.distMap) + alpha)
        
        self.SumPhi = np.sum(self.Phi, axis=0)
    
    def dispElectrode(self, ax, s=100, fontsize=5, fontcolor='w'):
        for ei in range(self.eNum):
            ax.scatter(self.eXY[ei,0], self.eXY[ei,1], s=s, c='darkslategrey')
            ax.text(self.eXY[ei,0]-12, self.eXY[ei,1]-5, ("E%d" % (ei+1)), fontsize=fontsize, color=fontcolor)
        ax.set_aspect('equal',adjustable='box')
    
    def dispFingerLayout(self, ax, lw=1):
        arcObj = Arc([self.centerX, self.centerY], 2*self.cRadius, 2*self.cRadius, angle=0, theta1=0.0, 
                theta2=180.0, color='turquoise', lw=lw)
        ax.add_patch(arcObj)
        ax.plot([self.centerX-self.cRadius, self.centerX-self.cRadius], [self.bottomY, self.centerY], color='turquoise', lw=lw)
        ax.plot([self.centerX+self.cRadius, self.centerX+self.cRadius], [self.bottomY, self.centerY], color='turquoise', lw=lw)
        ax.plot([self.centerX-self.cRadius, self.centerX+self.cRadius], [self.bottomY, self.bottomY], color='turquoise', lw=lw)
        ax.set_aspect('equal',adjustable='box')
        
    def dispFingerSurface(self, ax):
        ax.plot(self.gridXY[:,0], self.gridXY[:,1], color='turquoise', lw=0.1)
        
    def constructMap(self, eValue):
        mapValue = np.matmul(eValue, self.Phi) / self.SumPhi
        return mapValue
        
    def dispMaps(self, ax, mapValues, xShift=0, cbarax=None, cmap=cm.get_cmap('Greys', 100), unifyRange=[0, 1], 
                 s=0.1, dispOutline=False):      
        frameNum = len(mapValues)
        
        for i in range(frameNum):
            scplt = ax.scatter(xShift*i+self.gridXY[:,0], self.gridXY[:,1], c=mapValues[i], cmap=cmap, s=s, norm=None, 
                       vmin=unifyRange[0], vmax=unifyRange[1])

        ax.set_aspect('equal',adjustable='box')

        ax.set_xlim([self.centerX-self.cRadius, xShift*i+self.centerX+self.cRadius])
        ax.set_ylim([self.bottomY, self.centerY + self.cRadius + 10])
        
        ax.axis('off')
        
        if cbarax is None:
            cbarax = ax
            
        cbar = plt.colorbar(scplt, ax=cbarax, fraction=0.05, pad=0.25, aspect=8)
        cbar.outline.set_visible(False)
        cbar.ax.get_yaxis().labelpad = 15
        cbar.ax.set_ylabel(r'$\Delta_{\mathrm{Impedance}} (\Omega$)', rotation=270)
        
        if dispOutline:
            self.dispFingerLayout(ax)
        
        return cbar
        
    def mapFromCubicInterp(self, ax, eValue, dispNode=False): # (Obsolete) Interpolation cubically between electrodes 
        grid_z = griddata(self.eXY, eValue, (self.grid_x, self.grid_y), method='cubic')
        
        ax.imshow(grid_z.T, extent=self.extent, origin='lower', cmap='gray')
        
        if dispNode:
            self.dispElectrode(ax)
            
        ax.set_aspect('equal',adjustable='box')
        ax.axis('off')

In [ ]:
''' Visualize real data and Biotac electrode impedance map: Charge time = 0.1, 0.4, 1.2 secs '''
measureDataPath = ".\Data2022.06.26"
root = 'Act60mm1.5mL'; 

'''---------------------------------------------------- 1.2 sec ----------------------------------------------------'''
fileName = 'v100c1200d4000t1.btd.csv' 
btData = loadDataSegment(measureDataPath, root, fileName, lpFreq=0)
btData['t'] = btData['t'] - 6.87999

tind = np.arange(687, 860, 20)
fig1, fig1cbar, fig2, ax1 = generateMapResult(tind, btData, yMax=4, unifyRange=[-25, 25])
ax1.set_yticks([0, 2.5, 5]);

''' To plot all waveforms in one axis '''
Act60v100c1200 = {}
Act60v100c1200['t'] = btData['t']
Act60v100c1200['pDC'] = btData['pDC']
Act60v100c1200['tind'] = tind

'''---------------------------------------------------- 0.4 sec ----------------------------------------------------'''
fileName = 'v100c400d4000t1.btd.csv' 
btData = loadDataSegment(measureDataPath, root, fileName, lpFreq=0)
btData['t'] = btData['t'] - 5.70

tind = np.arange(569,630, 7)
fig1, fig1cbar, fig2, ax1 = generateMapResult(tind, btData, yMax=4, unifyRange=[-16, 16])
ax1.set_yticks([0, 2.5, 5]);

''' To plot all waveforms in one axis '''
Act60v100c400 = {}
Act60v100c400['t'] = btData['t']
Act60v100c400['pDC'] = btData['pDC']
Act60v100c400['tind'] = tind

'''---------------------------------------------------- 0.1 sec ----------------------------------------------------'''
fileName = 'v100c100d4000t1.btd.csv' 
btData = loadDataSegment(measureDataPath, root, fileName, lpFreq=0)
btData['t'] = btData['t'] - 1.24999

tind = np.arange(124, 151, 3)
fig1, fig1cbar, fig2, ax1 = generateMapResult(tind, btData, yMax=4, unifyRange=[-10, 10])
ax1.set_yticks([0, 2.5, 5]);

''' To plot all waveforms in one axis '''
Act60v100c100 = {}
Act60v100c100['t'] = btData['t']
Act60v100c100['pDC'] = btData['pDC']
Act60v100c100['tind'] = tind

In [ ]:
''' To plot all waveforms in one axis for Act60v100c1200, Act60v100c400, Act60v100c100'''
def plotPressureSignal(dictin, ax, tindMargin=50, tShfit=0, yMax=5):
    indRange = [dictin['tind'][0]-tindMargin, dictin['tind'][-1]+tindMargin]
    ydata = dictin['pDC'][indRange[0]:indRange[1]]
    ydata = ydata - ydata[0]
 
    frameNum = len(dictin['tind'])
    for i in range(frameNum):
        ax.plot(dictin['t'][[dictin['tind'][i], dictin['tind'][i]]] + tShfit, [0, yMax], 'tab:grey', lw=0.5)
        
    ax.plot(dictin['t'][indRange[0]:indRange[1]] + tShfit, ydata) 
    ax.set_ylim([-0.1, yMax])

def plotCtrlSignal(ax, tOn=100, tMargin=200, tShift=0):
    segLen = 1600
    y = np.zeros((segLen,1))
    y[tMargin:tMargin+tOn] = 1
    x = np.arange(segLen)
    ax.plot((x + tShift - tMargin)/1000, y, 'tab:grey')
    
    
fig0, axes = plt.subplots(2, 1, dpi=300, figsize=(3.2, 0.6))
plotPressureSignal(Act60v100c100, axes[0])
plotPressureSignal(Act60v100c400, axes[0], tShfit=1.6)
plotPressureSignal(Act60v100c1200, axes[0], tShfit=3.6)

axes[0].set_xlim([-0.2, 5.8])

plotCtrlSignal(axes[1], tOn=100, tShift=0)
plotCtrlSignal(axes[1], tOn=400, tShift=1600)
plotCtrlSignal(axes[1], tOn=1200, tShift=3600)

axes[1].set_xlim([-0.2, 5.8])
